# Imports

In [16]:
import pandas as pd; import os; from analysis_helpers import *; import warnings
warnings.filterwarnings('ignore')

# Some Functions

In [25]:
def list_logs(data_dir):
    '''
    input : path to participant data directory  (str)
    output: participant's full behavioral log   (df)
    '''
            
    participant = []
    for f in os.listdir(data_dir):

        if f !='--1.log' and ('.log' in f):

            with open(data_dir+f) as file:

                    lines = file.read().splitlines()
                    lines = [[lines[x]] for x in range(0, len(lines))]
                    #lines.insert(0,['DATA'])

                    log_file = pd.DataFrame(lines).reset_index()

                    log_file['Subject'] = sub_dir.split('_')[0]
                    log_file['Run'] = int(f.split('.')[-2][-1])
                    log_file['TIME'],log_file['WARNING'],log_file['MESSAGE'] = log_file[0].str.split('\t', 4).str
                    log_file = log_file.fillna('NAN VALUE')
                    log_file['TIME'] = log_file['TIME'].str.replace(' ','')
                    log_file['TIME'] = log_file['TIME'].astype(float, errors='ignore')

                    a = log_file[log_file[0].str.contains('current')][0].str.split(' ', expand=True)
                    log_file['TIME'] = log_file[['TIME']].applymap(lambda x: np.nan if isinstance(x, str) else float(x))
                    #log_file['TIME'] = log_file['TIME'] + float(a[4]) - float(a[0])

#                             utc = log_file[0].str.contains('current')['MESSAGE'].split(' ')[-1]
#                             psycho = log_file[0].str.contains('current')['TIME']

                    participant.append(log_file)

    participant = pd.concat(participant)
    return(participant)

# Data Compile
Organize behavioral data from each participant into one dataframe for the whole experiment.

In [71]:
# Compile data from all exps
# add attention labels to memory stim
# correct timings for group1 - safest estimate
# check and add gaze data
# output dataframe with attn labels, timing, and gaze data

# run behavioral checks and further gaze checks

a,b = '../sustained_attention_experiment/data/','../variable_attention_experiment/data/'
# data_dirs = [a+'group1', a+'group2', b+'group1', b+'group2']
data_dirs = [b+'group1', b+'group2']
filenames = [x+'_partial_new.csv' for x in data_dirs]

for data,file in zip(data_dirs, filenames):
    
    sub_list = []
    
    # for each subject in this directory
    for sub_dir in os.listdir(data):
        
        if sub_dir != '.DS_Store' and sub_dir != 'README.md':
            
            print(data)
            print(sub_dir)
            print()
            
            # add attention labels to memory stim 
            subject = add_level(sum_pd(data + '/' + sub_dir))
            
            # TIMING CORRRECTIONS (for gaze)
            if data[-1] == '1':
               
                # github Issue #83, Check B: times in behavioral csv's (Group1) imprecise by fractions of a second
                # correct for those timing issues (for gaze analysis), using safest estimates 
                
                # SUSTAINED: cued composite starts .0178 seconds earlier; VAR : .0167 earlier 
                # SUSTAINED: composites disappear 0.0322 secs later; VARIABLE: .0359 secs 
                subject.loc[subject['Trial Type']=='Presentation','Stimulus Onset'] = subject[subject['Trial Type']=='Presentation']['Stimulus Onset'] - .0178
                subject.loc[subject['Trial Type']=='Presentation','Stimulus End']   = subject[subject['Trial Type']=='Presentation']['Stimulus End']   + .0359
                  
                # SUST: mem images display 0.0179 earlier; VAR: .0142
                # SUST: mem images disappear .259 secs later; VAR: .137
                subject.loc[subject['Trial Type']=='Memory','Stimulus Onset'] = subject[subject['Trial Type']=='Memory']['Stimulus Onset'] - .0179
                subject.loc[subject['Trial Type']=='Memory','Stimulus End']   = subject[subject['Trial Type']=='Memory']['Stimulus End']   +.259
                
                # NOTE: no correction for button press differences, as they averaged .001 seconds or less

                
                if data == '../variable_attention_experiment/data/group1':
                
                    # EXP 2 cue corrections ---------------
                    # github Issue #83, Check E: some Group1, Var Exp valid cues marked as invalid
                    
                    # correct cue labels Var Exp, Group 1
                    for run in subject['Run'].unique():
                        first_cue = list(subject[(subject['Run']==run) & (subject['Trial Type']=='Presentation')]['Cued Side'])
                        if subject[(subject['Run']==run) & (subject['Cue Validity']==0) & (subject['Cued Side'] != first_cue[0])].shape[0]>0:
                            subject.loc[(subject['Run']==run) & (subject['Cue Validity']==0) & (subject['Cued Side'] != first_cue[0]), 'Cue Validity'] = 1
                        
                        
            if data[-1] == '2':
                subject_log = list_logs(data + '/' + sub_dir+'/')
                subject_log['Subject'] = pd.to_numeric(subject_log['Subject'])
                subject_log = subject_log.sort_values(by=['Subject','Run','TIME'])
                subject = subject.sort_values(by=['Subject','Run'])
                
                # now, extract desired stim on and off times
                composite_onsets  = subject_log[subject_log[0].str.contains('COMPOSITES ON')]
                composite_offsets = subject_log[subject_log[0].str.contains('COMPOSITES OFF')]
                attention_on = composite_offsets 
                
                subject.loc[subject['Trial Type']=='Presentation', 'Stimulus Onset'] = list(composite_onsets['TIME'])
                subject.loc[subject['Trial Type']=='Presentation', 'Stimulus End'  ] = list(composite_offsets['TIME'])
                subject.loc[subject['Trial Type']=='Presentation','Attention Response Time (s)'] = subject[subject['Trial Type']=='Presentation']['Attention Response Time (s)'] - subject[subject['Trial Type']=='Presentation']['Stimulus End']
                
                memory_onsets  = list(subject_log[ (subject_log[0].str.contains('MEMORY ON')) & (subject_log[0].str.contains('FLIP')) ]['TIME'])
                memory_offsets = list(subject_log[ (subject_log[0].str.contains('MEMORY OFF')) & (subject_log[0].str.contains('FLIP')) ]['TIME'])
                
                subject.loc[subject['Trial Type']=='Memory', 'Stimulus Onset'] = memory_onsets
                subject.loc[subject['Trial Type']=='Memory', 'Stimulus End'  ] = memory_offsets
                
            # add ON and OFF stim times for group 2
                
                
            # GROUP 2 -- attention probe reaction time conversions
                
                # attention reaction times pulled from log file use psychopy timeline
                # switching to time from stim onset (reaction time)
                
                
            # Add gaze data (internal)
            # renumber the subject (internal)
                

            sub_list.append(subject)
                
            
    exp_raw = pd.concat(sub_list)
    exp_raw.to_csv(file)


../variable_attention_experiment/data/group1
8_2019_Feb_28

../variable_attention_experiment/data/group1
15_2019_Apr_01

../variable_attention_experiment/data/group1
1_2019_Feb_21

../variable_attention_experiment/data/group1
25_2019_Apr_16

../variable_attention_experiment/data/group1
12_2019_Mar_27

../variable_attention_experiment/data/group1
4_2019_Feb_21

../variable_attention_experiment/data/group1
28_2019_Apr_23

../variable_attention_experiment/data/group1
19_2019_Apr_01

../variable_attention_experiment/data/group1
21_2019_Apr_09

../variable_attention_experiment/data/group1
33_2019_Apr_26

../variable_attention_experiment/data/group1
7_2019_Feb_28

../variable_attention_experiment/data/group1
26_2019_Apr_16

../variable_attention_experiment/data/group1
14_2019_Mar_29

../variable_attention_experiment/data/group1
2_2019_Feb_21

../variable_attention_experiment/data/group1
16_2019_Apr_01

../variable_attention_experiment/data/group1
27_2019_Apr_22

../variable_attention_experim

In [70]:
composite_onsets['Run'].unique()


array([0, 1, 2, 3, 4, 5, 6])

In [52]:
subject.loc[(subject['Run']==run) & (subject['Cue Validity']==0) & (subject['Cued Side'] != first_cue)].shape

(0, 26)

In [4]:
# Compile the behavioral and gaze data for both exps, Groups 1 and 2

    # for each subj:
        
        # compile behavioral
        
            # pull from log files and convert times as needed
        
        # check gaze data (file for each, data for all times needed, units, etc)
        
        # check for redundancies
        
        # do gaze calcs and add to dataframe
        
        
        # save out compiled full csv in this subject's dir (under temp)
        

In [ ]:
            # GROUP 2 -- attention probe reaction time conversions
            if data_dirs[-1] == '2':
                
                # attention reaction times pulled from log file use psychopy timeline
                # switching to time from stim onset (reaction time)
                

In [5]:
# Compile the behavioral data for both exps, Groups 1 and 2

# Complete all necessary checks

# 

# Produce a single dataframe with all of the necessary data, labeled by: 
# experiment and group number



data_dirs = ['../sustained_attention_experiment/data',
             '../variable_attention_experiment/data',
            '',
            '']



filenames = ['../parsed_data/behavioral_data_sustained.csv', 
             '../parsed_data/behavioral_data_variable.csv']

for data,file in zip(data_dirs, filenames):
    
    sub_list = []
    
    for sub_dir in os.listdir(data):
        sub_list.append(add_level(sum_pd(data + '/' + sub_dir)))

    exp_raw = pd.concat(sub_list)
    exp_raw.to_csv(file)

NotADirectoryError: [Errno 20] Not a directory: '../sustained_attention_experiment/data/.DS_Store'